In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from keras import layers, models, optimizers, regularizers
from keras.applications import EfficientNetB0
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib.pyplot as plt

In [2]:
# test if running the GPU version of tensorflow
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# loads directory of 3 datasets
dir_train = pd.read_csv('dataset/EuroSAT/train.csv')
dir_valid = pd.read_csv('dataset/EuroSAT/validation.csv')
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')


In [5]:
img_dir = pd.concat([dir_train, dir_valid], ignore_index=False)
img_dir = img_dir.iloc[:,1:-1].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:100, :] # limit sample size when testing
img_dir

(24300, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,Industrial/Industrial_453.jpg,4,Industrial
4,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...
24295,SeaLake/SeaLake_1943.jpg,9,SeaLake
24296,AnnualCrop/AnnualCrop_211.jpg,0,AnnualCrop
24297,Industrial/Industrial_1428.jpg,4,Industrial
24298,AnnualCrop/AnnualCrop_2571.jpg,0,AnnualCrop


In [6]:
# Select a random, balanced subset of images
num_samples_per_class = 500
class_names = img_dir['ClassName'].unique()
subset_idx = []

for class_name in class_names:
    class_indices = img_dir[img_dir['ClassName'] == class_name].index
    random_indices = np.random.choice(class_indices, num_samples_per_class, replace=False)
    subset_idx.extend(random_indices)

subset_img_dir = img_dir.loc[subset_idx]

In [7]:
# Load images and labels/classes
images = []
classes = []
# labels.typeof()

base_path = 'dataset/EuroSAT/'
for index, row in subset_img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])

# Normalize images
images = np.array(images) / 255.0
labels = pd.get_dummies(classes).values

In [8]:
images.shape

(5000, 224, 224, 3)

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Set up the data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
valid_data_generator = ImageDataGenerator()

In [10]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"

# hub_layer = hub.KerasLayer(model_path, trainable=False)

def create_model(optimizer="adam", learning_rate=0.001, num_dense_layers=1, dropout_rate=0.5):
    optimizers = {
        "adam": Adam(learning_rate=learning_rate),
        "rmsprop": RMSprop(learning_rate=learning_rate)
    }
    
    hub_layer = hub.KerasLayer(MODEL_PATH, trainable=False)
    
    model = keras.Sequential()
    model.add(keras.layers.InputLayer((224, 224, 3)))
    model.add(hub_layer)
    
    for _ in range(num_dense_layers):
        model.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizers[optimizer], metrics=["accuracy"])
    
    return model

batch_size = 16

model = KerasClassifier(build_fn=create_model, epochs=3, batch_size=batch_size, verbose=0)


C:\Users\Geran\AppData\Local\Temp\ipykernel_12560\3856091526.py:33: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=3, batch_size=batch_size, verbose=0)


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "dropout_rate": [0.5, 0.2],
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END ...................................dropout_rate=0.5; total time=  16.1s
[CV] END ...................................dropout_rate=0.5; total time=  16.4s
[CV] END ...................................dropout_rate=0.5; total time=  15.9s
[CV] END ...................................dropout_rate=0.2; total time=  16.1s
[CV] END ...................................dropout_rate=0.2; total time=  16.2s
[CV] END ...................................dropout_rate=0.2; total time=  15.9s


ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\keras\wrappers\scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
  File "c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\keras\wrappers\scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handl

In [ ]:
# Train the model using the best hyperparameters found during the search
batch_size = 32
epochs = 3
validation_steps = len(X_test) // batch_size

best_params = grid_result.best_params_

best_model = create_model(dropout_rate=best_params["dropout_rate"])
best_model.compile(loss="categorical_crossentropy")

history = best_model.fit(train_data_generator.flow(X_train, y_train, batch_size=batch_size), validation_data=valid_data_generator.flow(X_test, y_test, batch_size=batch_size), validation_steps=validation_steps, epochs=epochs)

Epoch 1/3
25/25 [==============================] - 50s 2s/step - loss: 1.7487 - val_loss: 1.1957
Epoch 2/3
25/25 [==============================] - 39s 2s/step - loss: 1.0096 - val_loss: 1.0134
Epoch 3/3
25/25 [==============================] - 40s 2s/step - loss: 0.8924 - val_loss: 0.8055


In [ ]:
grid_result.best_params_

{'dropout_rate': 0.5}